In [43]:
import sqlite3
#https://datatofish.com/create-database-python-using-sqlite3/
conn = sqlite3.connect('CourseData.db')  # You can create a new database by changing the name within the quotes
c = conn.cursor() # The database will be saved in the location where your 'py' file is saved

# Create table - IMPORT_Courses
c.execute('''DROP TABLE IF EXISTS IMPORT_COURSES''')
c.execute('''CREATE TABLE IMPORT_COURSES(
    [term] TEXT,
    [crn] TEXT,
    [catalog_id] TEXT,
    [section] TEXT,
    [credits] TEXT,
    [title] TEXT,
    [meetings] TEXT,
    [timecodes] TEXT,
    [primary_instructor] TEXT,
    [cap] INT,
    [act] INT, 
    [rem] INT,
    [instructor_fname] TEXT,
    [instructor_lname] TEXT,
    [semester] TEXT,
    [year] TEXT)'''
         )
          
# Create table - IMPORT_COURSE_MEETINGS
c.execute('''DROP TABLE IF EXISTS IMPORT_COURSE_MEETINGS''')
c.execute('''CREATE TABLE IMPORT_COURSE_MEETINGS (
    [crn] TEXT,
    [location] TEXT,
    [day] TEXT,
    [start] TEXT,
    [end] TEXT,
    [term] TEXT,
    [semester] TEXT,
    [year] TEXT,
    [BuildingName]    TEXT , 
    [BuildingCode] TEXT )'''
         )
        
# Create table - IMPORT_COURSE_CATALOG
c.execute('''DROP TABLE IF EXISTS IMPORT_COURSE_CATALOG''')

c.execute('''CREATE TABLE IMPORT_COURSE_CATALOG (
    [program_code] TEXT,
    [program_name] TEXT,
    [catalog_ID] TEXT,
    [course_title] TEXT,
    [credits] TEXT,
    [prereqs] TEXT,
    [coreqs] TEXT,
    [fees] TEXT,
    [attributes] TEXT,
    [description] TEXT,
    [CatalogYear] TEXT)'''
         )


   
conn.commit()

# Note that the syntax to create new tables should only be used once in the code (unless you dropped the table/s at the end of the code). 
# The [generated_id] column is used to set an auto-increment ID for each record
# When creating a new table, you can add both the field names as well as the field formats (e.g., Text)

conn.close()




In [44]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine("sqlite:///CourseData.db")
dbtable = ['IMPORT_COURSES' ,'IMPORT_COURSE_CATALOG' , 'IMPORT_COURSE_MEETINGS',
]
for i in dbtable :
    print ('Table :' ,i)
    tables = pd.read_sql_table(i,engine)
    display(tables.head())
#IMPORT_COURSE_CATALOG = pd.read_sql_table('IMPORT_COURSE_CATALOG',engine)
#display(IMPORT_COURSE_CATALOG.head())
#IMPORT_COURSE_MEETINGS = pd.read_sql_table('IMPORT_COURSE_MEETINGS',engine)
#display(IMPORT_COURSE_MEETINGS.head())

#df.to_sql('IMPORT_Course_Meetings', conn, if_exists='append', index=False)

Table : IMPORT_COURSES


,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem,instructor_fname,instructor_lname,semester,year


Table : IMPORT_COURSE_CATALOG


,program_code,program_name,catalog_ID,course_title,credits,prereqs,coreqs,fees,attributes,description,CatalogYear


Table : IMPORT_COURSE_MEETINGS


,crn,location,day,start,end,term,semester,year,BuildingName,BuildingCode


In [45]:

import pandas as pd
#creating coureses df
##https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe

semester_folders = ['Fall2014',
                    'Fall2015',
                    'Fall2016',
                    'Fall2017',
                    'Fall2018',
                    'Spring2015', 
                    'Spring2016',
                    'Spring2017',
                    'Spring2018',
                    'Spring2019',
                    'SpringBreak2017',
                    'Summer2015',
                    'Summer2016',
                    'Summer2017',
                    'Summer2018',
                    'Winter2015',
                    'Winter2016', 
                    'Winter2017',
                    'Winter2018']



# loop through the files and read them in with pandas
dataframes = []  # a list to hold all the individual pandas DataFrames
for semester in semester_folders:
    # the path to your csv file directory
    mycsvdir = 'SourceData/{}/courses.csv'.format(semester)
# read all the csv files in that directory (assuming they have the extension courses.csv)

    df = pd.read_csv(mycsvdir)
    dataframes.append(df)

# concatenate them all together
result = pd.concat(dataframes, ignore_index=True)

# print out to a new csv file
result.to_csv('courses_all.csv')
result.shape

(15937, 12)

In [46]:
#https://stackoverflow.com/questions/40950310/strip-trim-all-strings-of-a-dataframe

def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)



df_courses = pd.read_csv('courses_all.csv' ,header=0)
df_courses = trim_all_columns(df_courses)


df_courses[['instructor_fname','instructor_lname']] = df_courses['primary_instructor'].str.split('  ',expand=True,)
df_courses[['semester','year']] = df_courses['term'].str.split('2',expand=True,)
df_courses['year'] = '2'+df_courses['year']
# imput meetings and timecodes columns
df_courses.meetings.loc[df_courses.meetings == '[]'] = 'No meetings'
df_courses.timecodes.loc[df_courses.timecodes == '[]']= 'No time codes'

#cleaning the missing values in primary_instructor
primary_instructor_values = [
    
 '09/15-10/31',

 '09/02-12/30',

 '08/15-01/15',

 '09/02-12/19',

 '09/01-12/07',
 
 '05/21-08/17',
 '01/20-05/08',
 
 '01/20-04/30'

]


for i in primary_instructor_values :
    #print(df_courses.primary_instructor.loc[df_courses.primary_instructor == i].count())# = 'No meetings'
    df_courses.primary_instructor.loc[df_courses.primary_instructor == i] = 'TBA'
    
df_courses["instructor_lname"] = df_courses["instructor_lname"].fillna("TBA")

df_courses.pop('Unnamed: 0')

display(df_courses.head(2).T)
display(df_courses.describe().T)


,0,1
term,Fall2014,Fall2014
crn,70384,70385
catalog_id,AC 0011,AC 0011
section,C01,C02
credits,3.000,3.000
title,Introduction to Financial Accounting,Introduction to Financial Accounting
meetings,"[{'days': 'TF', 'times': '0800am-0915am', 'dat...","[{'days': 'TF', 'times': '0930am-1045am', 'dat..."
timecodes,['TF 0800am-0915am 09/02-12/08 DSB 105'],['TF 0930am-1045am 09/02-12/08 DSB 105']
primary_instructor,Michael P. Coyne,Michael P. Coyne
cap,0,0


,count,mean,std,min,25%,50%,75%,max
crn,15937.0,55172.299178,19308.879968,10893.0,35426.0,57145.0,74541.0,79215.0
cap,15937.0,17.992784,12.671906,0.0,10.0,19.0,25.0,500.0
act,15937.0,15.578591,10.107690,-1.0,7.0,15.0,23.0,90.0
rem,15937.0,2.477317,12.709781,-75.0,0.0,1.0,5.0,999.0


In [47]:
df_courses.isna().sum()

term                  0
crn                   0
catalog_id            0
section               0
credits               0
title                 0
meetings              0
timecodes             0
primary_instructor    0
cap                   0
act                   0
rem                   0
instructor_fname      0
instructor_lname      0
semester              0
year                  0
dtype: int64

In [48]:
conn = sqlite3.connect('CourseData.db')  # You can create a new database by changing the name within the quotes
c = conn.cursor() # The database will be saved in the location where your 'py' file is saved

df_courses.to_sql('IMPORT_COURSES', conn, if_exists='append', index=False)
conn.close()


In [49]:
df_courses.describe().T

,count,mean,std,min,25%,50%,75%,max
crn,15937.0,55172.299178,19308.879968,10893.0,35426.0,57145.0,74541.0,79215.0
cap,15937.0,17.992784,12.671906,0.0,10.0,19.0,25.0,500.0
act,15937.0,15.578591,10.107690,-1.0,7.0,15.0,23.0,90.0
rem,15937.0,2.477317,12.709781,-75.0,0.0,1.0,5.0,999.0


In [50]:

import pandas as pd
#creating course_meetings df
##https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe

semester_folders = ['Fall2014',
                    'Fall2015',
                    'Fall2016',
                    'Fall2017',
                    'Fall2018',
                    'Spring2015', 
                    'Spring2016',
                    'Spring2017',
                    'Spring2018',
                    'Spring2019',
                    'SpringBreak2017',
                    'Summer2015',
                    'Summer2016',
                    'Summer2017',
                    'Summer2018',
                    'Winter2015',
                    'Winter2016', 
                    'Winter2017',
                    'Winter2018']



# loop through the files and read them in with pandas
dataframes = []  # a list to hold all the individual pandas DataFrames
for semester in semester_folders:
    # the path to your csv file directory
    mycsvdir = 'SourceData/{}/course_meetings.csv'.format(semester)
# read all the csv files in that directory (assuming they have the extension course_meetings.csv)

    df = pd.read_csv(mycsvdir)
    dataframes.append(df)

# concatenate them all together
result = pd.concat(dataframes, ignore_index=True)

# print out to a new csv file
result.to_csv('course_meetings_all.csv')



In [51]:

df_course_meetings = pd.read_csv('course_meetings_all.csv' ,header=0)
df_course_meetings = trim_all_columns(df_course_meetings)

df_course_meetings[['semester','year']] = df_course_meetings['term'].str.split('2',expand=True,)
df_course_meetings['year'] = '2'+df_course_meetings['year']
print(df_course_meetings.shape)
df_b = pd.read_csv('buildings2.csv' ,header=0)
df_b = trim_all_columns(df_b)

df_course_meetings_buildings = df_course_meetings.merge(df_b  ,left_on='location', right_on='location')


df_course_meetings_buildings.pop('Unnamed: 0_y')
df_course_meetings_buildings.pop('Unnamed: 0_x')
print(df_course_meetings_buildings.shape)

(284907, 9)
(284907, 10)


In [52]:
df_course_meetings_buildings.head()

,term,crn,location,day,start,end,semester,year,BuildingName,BuildingCode
0,Fall2014,70384,DSB 105,T,2014-09-02T08:00:00,2014-09-02T09:15:00,Fall,2014,Dolan School of Business,DSB
1,Fall2014,70384,DSB 105,F,2014-09-05T08:00:00,2014-09-05T09:15:00,Fall,2014,Dolan School of Business,DSB
2,Fall2014,70384,DSB 105,T,2014-09-09T08:00:00,2014-09-09T09:15:00,Fall,2014,Dolan School of Business,DSB
3,Fall2014,70384,DSB 105,F,2014-09-12T08:00:00,2014-09-12T09:15:00,Fall,2014,Dolan School of Business,DSB
4,Fall2014,70384,DSB 105,T,2014-09-16T08:00:00,2014-09-16T09:15:00,Fall,2014,Dolan School of Business,DSB


In [53]:
conn = sqlite3.connect('CourseData.db')  # You can create a new database by changing the name within the quotes
c = conn.cursor() # The database will be saved in the location where your 'py' file is saved

df_course_meetings_buildings.to_sql('IMPORT_COURSE_MEETINGS', conn, if_exists='append', index=False)
conn.close()

In [54]:
#https://github.com/christopherhuntley/ba510-course-data-project/blob/example/ETL_CourseData.ipynb
conn = sqlite3.connect('CourseData.db')  # You can create a new database by changing the name within the quotes
c = conn.cursor() # The database will be saved in the location where your 'py' file is saved


catalog_years = ['2017_2018', '2018_2019']

for cat_year in catalog_years:
    filepath = 'SourceData/Catalogs/CourseCatalog{}.csv'.format(cat_year)
    data = pd.read_csv(filepath)
    data['CatalogYear'] = cat_year
    data.to_sql('IMPORT_COURSE_CATALOG',conn,if_exists='append',index=False)
conn.close()


# Checking the Imported data Tables After Loading the Data:


In [55]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine("sqlite:///CourseData.db")
dbtable = ['IMPORT_COURSES' ,'IMPORT_COURSE_CATALOG' , 'IMPORT_COURSE_MEETINGS',
          
]
for i in dbtable :
    print ('Table :' ,i)
    tables = pd.read_sql_table(i,engine)
    display(tables.head())




Table : IMPORT_COURSES


,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem,instructor_fname,instructor_lname,semester,year
0,Fall2014,70384,AC 0011,C01,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0800am-0915am', 'dat...",['TF 0800am-0915am 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31,Michael P.,Coyne,Fall,2014
1,Fall2014,70385,AC 0011,C02,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0930am-1045am', 'dat...",['TF 0930am-1045am 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31,Michael P.,Coyne,Fall,2014
2,Fall2014,70382,AC 0011,C03,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '1230pm-0145pm', 'dat...",['TF 1230pm-0145pm 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31,Michael P.,Coyne,Fall,2014
3,Fall2014,70291,AC 0011,C04,3.000,Introduction to Financial Accounting,"[{'days': 'MR', 'times': '1100am-1215pm', 'dat...",['MR 1100am-1215pm 09/02-12/08 DSB 111'],Rebecca I. Bloch,0,29,-29,Rebecca I.,Bloch,Fall,2014
4,Fall2014,70350,AC 0011,C05,3.000,Introduction to Financial Accounting,"[{'days': 'MR', 'times': '1230pm-0145pm', 'dat...",['MR 1230pm-0145pm 09/02-12/08 DSB 111'],Rebecca I. Bloch,0,30,-30,Rebecca I.,Bloch,Fall,2014


Table : IMPORT_COURSE_CATALOG


,program_code,program_name,catalog_ID,course_title,credits,prereqs,coreqs,fees,attributes,description,CatalogYear
0,AN,Asian Studies,AN 0301,Independent Study,1-3 Credits,None,None,None,None,Students undertake an individualized program o...,2017_2018
1,AN,Asian Studies,AN 0310,Asian Studies Seminar,3 Credits,None,None,None,None,This seminar examines selected topics concerni...,2017_2018
2,BU,Business,BU 0211,Legal Environment of Business,3 Credits,Junior standing.,None,None,None,This course examines the broad philosophical a...,2017_2018
3,BU,Business,BU 0220,Environmental Law and Policy,3 Credits,None,None,None,"EVME Environmental Studies Major Elective, EVP...",This course surveys issues arising out of fede...,2017_2018
4,BU,Business,BU 0311,"The Law of Contracts, Sales, and Property",3 Credits,BU 0211.,None,None,None,This course examines the components of common ...,2017_2018


Table : IMPORT_COURSE_MEETINGS


,crn,location,day,start,end,term,semester,year,BuildingName,BuildingCode
0,70384,DSB 105,T,2014-09-02T08:00:00,2014-09-02T09:15:00,Fall2014,Fall,2014,Dolan School of Business,DSB
1,70384,DSB 105,F,2014-09-05T08:00:00,2014-09-05T09:15:00,Fall2014,Fall,2014,Dolan School of Business,DSB
2,70384,DSB 105,T,2014-09-09T08:00:00,2014-09-09T09:15:00,Fall2014,Fall,2014,Dolan School of Business,DSB
3,70384,DSB 105,F,2014-09-12T08:00:00,2014-09-12T09:15:00,Fall2014,Fall,2014,Dolan School of Business,DSB
4,70384,DSB 105,T,2014-09-16T08:00:00,2014-09-16T09:15:00,Fall2014,Fall,2014,Dolan School of Business,DSB


# Creating the ERD tables:

In [56]:
import sqlite3
#https://datatofish.com/create-database-python-using-sqlite3/
conn = sqlite3.connect('CourseData.db')  # You can create a new database by changing the name within the quotes
c = conn.cursor() # The database will be saved in the location where your 'py' file is saved




c.execute('''DROP TABLE IF EXISTS PROGRAMS''')

c.execute('''CREATE TABLE PROGRAMS ( 
    [PID] INTEGER PRIMARY KEY,
    [program_code] TEXT NOT NULL,
    [program_name] TEXT NOT NULL)''')

c.execute('''DROP TABLE IF EXISTS FACULTY''')

c.execute('''CREATE TABLE FACULTY ( 
    [FID] INTEGER PRIMARY KEY,
    [instructor_fname] TEXT NOT NULL,
    [instructor_lname] TEXT NOT NULL)''')

c.execute('''DROP TABLE IF EXISTS LOCATION''')

c.execute('''CREATE TABLE LOCATION ( 
    [LID] INTEGER PRIMARY KEY,
    [BuildingName] TEXT,
    [BuildingCode] TEXT,
    [location] TEXT)''')

c.execute('''DROP TABLE IF EXISTS COURSE''')

c.execute('''CREATE TABLE COURSE ( 
    [CID] INTEGER PRIMARY KEY,
    [PID] INTEGER NOT NULL,
    [CatalogYear] TEXT NOT NULL,
    [catalog_ID] TEXT NOT NULL,
    [course_title] TEXT NOT NULL,
    [credits] TEXT NOT NULL,
    [prereqs] TEXT,
    [coreqs] TEXT,
    [fees] text,
    [attributes] TEXT,
    [description] TEXT,
    FOREIGN KEY (PID) REFERENCES PROGRAMS (PID)
)''')


c.execute('''DROP TABLE IF EXISTS COURSE_OFFERING''')

c.execute('''CREATE TABLE COURSE_OFFERING ( 
    [COID] INTEGER PRIMARY KEY,
    [CID] INTEGER,
    [FID] INTEGER NOT NULL,
    [CRN] TEXT NOT NULL,
    [SEMESTER] TEXT NOT NULL,
    [YEAR] INTEGER NOT NULL,
    [TITLE] TEXT NOT NULL,
    [CATALOG_ID] TEXT,
    [CatalogYear] TEXT,
    [MEETINGS] TEXT,
    [TIMECODES] TEXT,
    [SECTION] TEXT NOT NULL,
    [CAP] INTEGER NOT NULL,
    [ACT] INTEGER NOT NULL,
    [REM] INTEGER NOT NULL,
    [instructor_lname] TEXT,
    [instructor_fname] TEXT,
    FOREIGN KEY (CID) REFERENCES COURSE (CID)
    FOREIGN KEY (FID) REFERENCES FACULTY (FID)
)''')


c.execute('''DROP TABLE IF EXISTS CLASS_MEETINGS''')

c.execute('''CREATE TABLE CLASS_MEETINGS ( 
    [CMID] INTEGER PRIMARY KEY,
    [COID] INTEGER,
    [LID] INTEGER,
    [DAY] TEXT,
    [START] TEXT,
    [END] TEXT,
    [CRN] TEXT,
    [SEMESTER] TEXT NOT NULL,
    [YEAR] INTEGER NOT NULL,
    FOREIGN KEY (LID) REFERENCES LOCATION (LID)
    FOREIGN KEY (COID) REFERENCES COURSE_OFFERING (COID)
)''')


c.execute('''DROP TABLE IF EXISTS CATALOG_YEAR''')

c.execute('''CREATE TABLE CATALOG_YEAR (
    [CatalogYear] TEXT,
    [YEAR] TEXT,
    [SEMESTER] TEXT
)''')



                 
conn.commit()

# Note that the syntax to create new tables should only be used once in the code (unless you dropped the table/s at the end of the code). 
# The [generated_id] column is used to set an auto-increment ID for each record
# When creating a new table, you can add both the field names as well as the field formats (e.g., Text)

conn.close()




# Populate the ERD Tables from The Imported Data Tables

In [57]:
import sqlite3
#https://datatofish.com/create-database-python-using-sqlite3/
conn = sqlite3.connect('CourseData.db')  # You can create a new database by changing the name within the quotes
c = conn.cursor() # The database will be saved in the location where your 'py' file is saved

c.execute('''INSERT INTO CATALOG_YEAR (Semester,Year,CatalogYear) VALUES 
('Fall','2014','2014_2015'),('Winter','2015','2014_2015'),
('Spring','2015','2014_2015'),('Summer','2015','2014_2015'),
('Fall','2015','2015_2016'),('Winter','2016','2015_2016'),
('Spring','2016','2015_2016'),('Summer','2016','2015_2016'),
('Fall','2016','2016_2017'),('Winter','2017','2016_2017'),
('Spring','2017','2016_2017'),('SpringBreak','2017','2016_2017'),
('Summer','2017','2016_2017'),('Fall','2017','2017_2018'),
('Winter','2018','2017_2018'),('Spring','2018','2017_2018'),
('Summer','2018','2017_2018'),('Fall','2018','2018_2019'),
('Winter','2019','2018_2019'),('Spring','2019','2018_2019')'''
         )



c.execute('''INSERT INTO PROGRAMS (program_code,program_name) 
SELECT DISTINCT program_code,program_name
FROM IMPORT_COURSE_CATALOG''')

c.execute('''INSERT INTO FACULTY (instructor_lname, instructor_fname)
SELECT DISTINCT instructor_lname, instructor_fname
FROM IMPORT_COURSES''')

c.execute('''INSERT INTO LOCATION (BuildingName,BuildingCode,Location) 
SELECT DISTINCT BuildingName,BuildingCode,location
FROM IMPORT_COURSE_MEETINGS''')

c.execute('''INSERT INTO COURSE (PID,CatalogYear,Catalog_ID,Course_Title,Credits,Prereqs,Coreqs,Fees,Attributes,Description)
SELECT DISTINCT PID,CatalogYear,Catalog_ID,Course_Title,Credits,Prereqs,Coreqs,Fees,Attributes,Description
FROM IMPORT_COURSE_CATALOG
    JOIN PROGRAMS ON (PROGRAMS.program_code=IMPORT_COURSE_CATALOG.program_code)''')

c.execute('''INSERT INTO COURSE_OFFERING (CID,FID,CRN,Semester,Year,Title,Catalog_ID, CatalogYear,Meetings,Timecodes,Section,Cap,Act,Rem,instructor_lname, instructor_fname)
SELECT DISTINCT CID,FID,CRN,Semester,Year,Title, catalog_ID, CatalogYear,Meetings,Timecodes,Section,Cap,Act,Rem,instructor_lname, instructor_fname
FROM IMPORT_COURSES
    LEFT JOIN CATALOG_YEAR using( semester ,year)
    LEFT JOIN COURSE using( CatalogYear , catalog_ID)
    LEFT JOIN FACULTY using (instructor_lname ,instructor_fname)''')


c.execute('''INSERT INTO CLASS_MEETINGS (COID,LID,CRN,Day,Start,End,SEMESTER,YEAR)
SELECT DISTINCT COURSE_OFFERING.COID,LOCATION.LID,COURSE_OFFERING.CRN,Day,Start,End,COURSE_OFFERING.SEMESTER,COURSE_OFFERING.YEAR
FROM IMPORT_COURSE_MEETINGS
    JOIN COURSE_OFFERING USING (SEMESTER,YEAR, CRN)
    LEFT JOIN LOCATION ON (IMPORT_COURSE_MEETINGS.Location = LOCATION.location)''')


#c.execute('''''')





                 
conn.commit()

# Note that the syntax to create new tables should only be used once in the code (unless you dropped the table/s at the end of the code). 
# The [generated_id] column is used to set an auto-increment ID for each record
# When creating a new table, you can add both the field names as well as the field formats (e.g., Text)

conn.close()


# checking The Populate the ERD Tables :

In [58]:

import pandas as pd
from sqlalchemy import create_engine
engine = create_engine("sqlite:///CourseData.db")
dbtable = ['PROGRAMS','FACULTY' , 'LOCATION' , 'COURSE' ,
            'CLASS_MEETINGS', 'CATALOG_YEAR' ,'COURSE_OFFERING'
]
for i in dbtable :
    print ('Table :' ,i)
    tables = pd.read_sql_table(i,engine)
    display(tables.head())
    print("missing if any :")
    print(tables.isna().sum())


Table : PROGRAMS


,PID,program_code,program_name
0,1,AN,Asian Studies
1,2,BU,Business
2,3,BL,Black Studies
3,4,BEN,Bioengineering
4,5,AR,Arabic


missing if any :
PID             0
program_code    0
program_name    0
dtype: int64
Table : FACULTY


,FID,instructor_fname,instructor_lname
0,1,Michael P.,Coyne
1,2,Rebecca I.,Bloch
2,3,Paul,Caster
3,4,Jo Ann,Drusbosky
4,5,Arleen N.,Kardos


missing if any :
FID                 0
instructor_fname    0
instructor_lname    0
dtype: int64
Table : LOCATION


...

missing if any :
CMID        0
COID        0
LID         0
DAY         0
START       0
END         0
CRN         0
SEMESTER    0
YEAR        0
dtype: int64
Table : CATALOG_YEAR


,CatalogYear,YEAR,SEMESTER
0,2014_2015,2014,Fall
1,2014_2015,2015,Winter
2,2014_2015,2015,Spring
3,2014_2015,2015,Summer
4,2015_2016,2015,Fall


missing if any :
CatalogYear    0
YEAR           0
SEMESTER       0
dtype: int64
Table : COURSE_OFFERING


,COID,CID,FID,CRN,SEMESTER,YEAR,TITLE,CATALOG_ID,CatalogYear,MEETINGS,TIMECODES,SECTION,CAP,ACT,REM,instructor_lname,instructor_fname
0,1,NaN,1,70384,Fall,2014,Introduction to Financial Accounting,AC 0011,2014_2015,"[{'days': 'TF', 'times': '0800am-0915am', 'dat...",['TF 0800am-0915am 09/02-12/08 DSB 105'],C01,0,31,-31,Coyne,Michael P.
1,2,NaN,1,70385,Fall,2014,Introduction to Financial Accounting,AC 0011,2014_2015,"[{'days': 'TF', 'times': '0930am-1045am', 'dat...",['TF 0930am-1045am 09/02-12/08 DSB 105'],C02,0,31,-31,Coyne,Michael P.
2,3,NaN,1,70382,Fall,2014,Introduction to Financial Accounting,AC 0011,2014_2015,"[{'days': 'TF', 'times': '1230pm-0145pm', 'dat...",['TF 1230pm-0145pm 09/02-12/08 DSB 105'],C03,0,31,-31,Coyne,Michael P.
3,4,NaN,2,70291,Fall,2014,Introduction to Financial Accounting,AC 0011,2014_2015,"[{'days': 'MR', 'times': '1100am-1215pm', 'dat...",['MR 1100am-1215pm 09/02-12/08 DSB 111'],C04,0,29,-29,Bloch,Rebecca I.
4,5,NaN,2,70350,Fall,2014,Introduction to Financial Accounting,AC 0011,2014_2015,"[{'days': 'MR', 'times': '1230pm-0145pm', 'dat...",['MR 1230pm-0145pm 09/02-12/08 DSB 111'],C05,0,30,-30,Bloch,Rebecca I.


missing if any :
COID                   0
CID                 9979
FID                    0
CRN                    0
SEMESTER               0
YEAR                   0
TITLE                  0
CATALOG_ID             0
CatalogYear            0
MEETINGS               0
TIMECODES              0
SECTION                0
CAP                    0
ACT                    0
REM                    0
instructor_lname       0
instructor_fname       0
dtype: int64


# Testing :

In [1]:
#%load_ext sql
%reload_ext sql
# initialize a database connection; may have to adjust username and password 

%sql sqlite:///CourseData.db

'Connected: @CourseData.db'

In [2]:
%%sql 
-- Record Counts for Catalog Meetings
SELECT 
    (SELECT Count(*) FROM IMPORT_COURSE_MEETINGS) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_COURSE_MEETINGS)) as 'DistinctCount';

 * sqlite:///CourseData.db
Done.


RawCount,DistinctCount
284907,284847


In [3]:
%%sql 
SELECT term,crn,location, day, start 
FROM IMPORT_COURSE_MEETINGS
GROUP BY term,crn,location, day, start
HAVING COUNT(*)>1;

 * sqlite:///CourseData.db
Done.


term,crn,location,day,start
Fall2014,73073,MCA 102,M,2014-09-08T18:30:00
Fall2014,73073,MCA 102,M,2014-09-15T18:30:00
Fall2014,73073,MCA 102,M,2014-09-22T18:30:00
Fall2014,73073,MCA 102,M,2014-09-29T18:30:00
Fall2014,73073,MCA 102,M,2014-10-06T18:30:00
Fall2014,73073,MCA 102,M,2014-10-20T18:30:00
Fall2014,73073,MCA 102,M,2014-10-27T18:30:00
Fall2014,73073,MCA 102,M,2014-11-03T18:30:00
Fall2014,73073,MCA 102,M,2014-11-10T18:30:00
Fall2014,73073,MCA 102,M,2014-11-17T18:30:00


In [4]:
%%sql

SELECT COUNT(*) FROM LOCATION;


 * sqlite:///CourseData.db
Done.


COUNT(*)
207


In [5]:
%%sql

SELECT PID, Catalog_ID FROM COURSE
LIMIT 10;


 * sqlite:///CourseData.db
Done.


PID,catalog_ID
1,AN 0301
1,AN 0310
2,BU 0211
2,BU 0220
2,BU 0311
2,BU 0312
2,BU 0320
2,BU 0391
3,BL 0101
3,BL 0398


In [6]:
%%sql
SELECT * FROM PROGRAMS
LIMIT 10;

 * sqlite:///CourseData.db
Done.


PID,program_code,program_name
1,AN,Asian Studies
2,BU,Business
3,BL,Black Studies
4,BEN,Bioengineering
5,AR,Arabic
6,AY,Anthropology
7,AS,American Studies
8,JA,Japanese
9,AH,Art History
10,AE,Applied Ethics


In [7]:
%%sql
SELECT FACULTY.instructor_fname,FACULTY.instructor_lname,Title,Catalog_id,Section,Semester,Year
FROM FACULTY
    JOIN COURSE_OFFERING USING(FID)
WHERE FACULTY.instructor_lname like '%Tao' and year = 2016
ORDER BY Year, Semester
LIMIT 20;

 * sqlite:///CourseData.db
Done.


instructor_fname,instructor_lname,TITLE,CATALOG_ID,SECTION,SEMESTER,YEAR
Jie,Tao,Introduction to Information Systems,IS 0100,03,Fall,2016
Jie,Tao,Introduction to Information Systems,IS 0100,04,Fall,2016
Jie,Tao,Data Mining and Applications,IS 0315,01,Fall,2016
Jie,Tao,Information Systems and Database Management,IS 0500,01,Fall,2016
Jie,Tao,Independent Study in Information Systems and Operations Management,IS 0598,JT,Fall,2016
Jie,Tao,Intro to Information Systems,IS 0100,D,Spring,2016
Jie,Tao,Data Mining and Business Intelligence,IS 0540,01,Spring,2016
Jie,Tao,Business Analytics and Big Data Mgmt,IS 0550,01,Spring,2016
Jie,Tao,Contemporary Topics: Advanced Data Mining Applications,IS 0585B,01,Summer,2016


In [8]:


%%sql
SELECT COID, CMID, START  ,CLASS_MEETINGS.year  ,CLASS_MEETINGS.SEMESTER ,CLASS_MEETINGS.CRN
FROM COURSE_OFFERING
    JOIN CLASS_MEETINGS USING (COID)
where CLASS_MEETINGS.YEAR = 2019 and CLASS_MEETINGS.SEMESTER == "Spring" and CLASS_MEETINGS.CRN = 39006
LIMIT 10;



 * sqlite:///CourseData.db
Done.


COID,CMID,START,YEAR,SEMESTER,CRN
13345,3140,2019-01-29T18:30:00,2019,Spring,39006
13345,3141,2019-02-05T18:30:00,2019,Spring,39006
13345,3142,2019-02-12T18:30:00,2019,Spring,39006
13345,3143,2019-02-26T18:30:00,2019,Spring,39006
13345,3144,2019-03-05T18:30:00,2019,Spring,39006
13345,3145,2019-03-12T18:30:00,2019,Spring,39006
13345,30636,2019-02-02T09:00:00,2019,Spring,39006
13345,30637,2019-02-16T09:00:00,2019,Spring,39006
13345,30638,2019-03-09T09:00:00,2019,Spring,39006


In [9]:
%%sql
SELECT CLASS_MEETINGS.YEAR ,CLASS_MEETINGS.SEMESTER,COID ,COUNT(COID) 
FROM COURSE_OFFERING JOIN CLASS_MEETINGS USING (COID)
where CLASS_MEETINGS.YEAR = 2019 and CLASS_MEETINGS.SEMESTER == "Spring" and CLASS_MEETINGS.CRN = 39006
GROUP BY CMID




 * sqlite:///CourseData.db
Done.


YEAR,SEMESTER,COID,COUNT(COID)
2019,Spring,13345,1
2019,Spring,13345,1
2019,Spring,13345,1
2019,Spring,13345,1
2019,Spring,13345,1
2019,Spring,13345,1
2019,Spring,13345,1
2019,Spring,13345,1
2019,Spring,13345,1


In [10]:
%%sql
-- There should be 4440 Catalog Entries, 15937 Course Offerings, and 284847 Course Meetings
SELECT 
    (SELECT Count(*) FROM COURSE) as CatalogCourses,
    (SELECT Count(*) FROM COURSE_OFFERING) as CourseOfferings,
    (SELECT Count(*) FROM CLASS_MEETINGS) as CourseMeetings;

 * sqlite:///CourseData.db
Done.


CatalogCourses,CourseOfferings,CourseMeetings
4440,15937,284847


In [11]:
%%sql
SELECT catalog_ID,program_code,program_name
FROM COURSE
    JOIN PROGRAMS USING (PID)
LIMIT 5;

 * sqlite:///CourseData.db
Done.


catalog_ID,program_code,program_name
AN 0301,AN,Asian Studies
AN 0310,AN,Asian Studies
BU 0211,BU,Business
BU 0220,BU,Business
BU 0311,BU,Business


In [12]:
%%sql
SELECT CLASS_MEETINGS.CRN,Day,Start,End,Catalog_id,Section,Meetings ,BuildingName ,Location as Room
FROM CLASS_MEETINGS
    JOIN COURSE_OFFERING USING (COID)
    JOIN LOCATION USING (LID)
LIMIT 5;

 * sqlite:///CourseData.db
Done.


CRN,DAY,START,END,CATALOG_ID,SECTION,MEETINGS,BuildingName,Room
70384,T,2014-09-02T08:00:00,2014-09-02T09:15:00,AC 0011,C01,"[{'days': 'TF', 'times': '0800am-0915am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",Dolan School of Business,DSB 105
70384,F,2014-09-05T08:00:00,2014-09-05T09:15:00,AC 0011,C01,"[{'days': 'TF', 'times': '0800am-0915am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",Dolan School of Business,DSB 105
70384,T,2014-09-09T08:00:00,2014-09-09T09:15:00,AC 0011,C01,"[{'days': 'TF', 'times': '0800am-0915am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",Dolan School of Business,DSB 105
70384,F,2014-09-12T08:00:00,2014-09-12T09:15:00,AC 0011,C01,"[{'days': 'TF', 'times': '0800am-0915am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",Dolan School of Business,DSB 105
70384,T,2014-09-16T08:00:00,2014-09-16T09:15:00,AC 0011,C01,"[{'days': 'TF', 'times': '0800am-0915am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",Dolan School of Business,DSB 105


In [15]:
import pandas as pd
#https://datatofish.com/create-database-python-using-sqlite3/
from sqlalchemy import create_engine

engine = create_engine("sqlite:///CourseData.db")
#pick a table to load into a df
table = 'COURSE_OFFERING'
#load the table into a df  and give a condition( where, join)
query = '''SELECT * FROM {} LIMIT 5'''.format(table)
tables = pd.read_sql_query(query,engine)
print("------------------------------------------------------------")
print("------------------------------------------------------------")
print("              tables shape :",tables.shape)
print("------------------------------------------------------------")
print("------------------------------------------------------------")
display(tables.head(2))
#load the whole table into a df
tables_all = pd.read_sql_table (table ,engine)
print("------------------------------------------------------------")
print("------------------------------------------------------------")
print("               tables_all shape: ",tables_all.shape)
print("------------------------------------------------------------")
print("------------------------------------------------------------")
tables_all.head(2)

------------------------------------------------------------
------------------------------------------------------------
              tables shape : (5, 17)
------------------------------------------------------------
------------------------------------------------------------


,COID,CID,FID,CRN,SEMESTER,YEAR,TITLE,CATALOG_ID,CatalogYear,MEETINGS,TIMECODES,SECTION,CAP,ACT,REM,instructor_lname,instructor_fname
0,1,None,1,70384,Fall,2014,Introduction to Financial Accounting,AC 0011,2014_2015,"[{'days': 'TF', 'times': '0800am-0915am', 'dat...",['TF 0800am-0915am 09/02-12/08 DSB 105'],C01,0,31,-31,Coyne,Michael P.
1,2,None,1,70385,Fall,2014,Introduction to Financial Accounting,AC 0011,2014_2015,"[{'days': 'TF', 'times': '0930am-1045am', 'dat...",['TF 0930am-1045am 09/02-12/08 DSB 105'],C02,0,31,-31,Coyne,Michael P.


------------------------------------------------------------
------------------------------------------------------------
               tables_all shape:  (15937, 17)
------------------------------------------------------------
------------------------------------------------------------


,COID,CID,FID,CRN,SEMESTER,YEAR,TITLE,CATALOG_ID,CatalogYear,MEETINGS,TIMECODES,SECTION,CAP,ACT,REM,instructor_lname,instructor_fname
0,1,NaN,1,70384,Fall,2014,Introduction to Financial Accounting,AC 0011,2014_2015,"[{'days': 'TF', 'times': '0800am-0915am', 'dat...",['TF 0800am-0915am 09/02-12/08 DSB 105'],C01,0,31,-31,Coyne,Michael P.
1,2,NaN,1,70385,Fall,2014,Introduction to Financial Accounting,AC 0011,2014_2015,"[{'days': 'TF', 'times': '0930am-1045am', 'dat...",['TF 0930am-1045am 09/02-12/08 DSB 105'],C02,0,31,-31,Coyne,Michael P.


In [16]:
%%sql
DELETE FROM IMPORT_COURSE_MEETINGS;
DELETE FROM IMPORT_COURSES;
DELETE FROM IMPORT_COURSE_CATALOG;


 * sqlite:///CourseData.db
284907 rows affected.
15937 rows affected.
4440 rows affected.


[]

In [17]:
%%sql
DROP TABLE IMPORT_COURSE_MEETINGS;
DROP TABLE IMPORT_COURSES;
DROP TABLE IMPORT_COURSE_CATALOG;


 * sqlite:///CourseData.db
Done.
Done.
Done.


[]

In [18]:
%%sql
vacuum;

 * sqlite:///CourseData.db
Done.


[]

In [19]:
conn = sqlite3.connect('CourseData.db') 
c = conn.cursor() 

#checking the db tables after dropping the import tables.

c.execute('SELECT name from sqlite_master where type= "table"')

print(c.fetchall())
conn.close()

[('PROGRAMS',), ('FACULTY',), ('LOCATION',), ('COURSE',), ('COURSE_OFFERING',), ('CLASS_MEETINGS',), ('CATALOG_YEAR',)]


In [21]:
import sqlite3
from sqlalchemy import create_engine
import pandas as pd
#https://datatofish.com/create-database-python-using-sqlite3/

#checking the db tables after dropping the import tables.

query = 'SELECT name from sqlite_master where type= "table"'
engine = create_engine("sqlite:///CourseData.db")

tables = pd.read_sql_query(query,engine)
tables = tables.rename(columns={"name": "DatabaseTables"})
tables

,DatabaseTables
0,PROGRAMS
1,FACULTY
2,LOCATION
3,COURSE
4,COURSE_OFFERING
5,CLASS_MEETINGS
6,CATALOG_YEAR


In [26]:
#checking the db tables after dropping the import tables.user_input  = input
query = 'SELECT name from sqlite_master where type= "table"'
engine = create_engine("sqlite:///CourseData.db")

tables = pd.read_sql_query(query,engine)
tables = tables.rename(columns={"name": "DatabaseTables"})
tables.DatabaseTables.values

array(['PROGRAMS', 'FACULTY', 'LOCATION', 'COURSE', 'COURSE_OFFERING',
       'CLASS_MEETINGS', 'CATALOG_YEAR'], dtype=object)

In [34]:
#checking the a table in dB for columns information: (id, name, type, notnull, default_value, primary_key)
table = "FACULTY"
query = "PRAGMA TABLE_INFO ({}) ".format (table)
engine = create_engine("sqlite:///CourseData.db")

table_info = pd.read_sql_query(query,engine)
table_info =table_info.rename(columns={"name": "ColumnName" ,"type": "DataType" , 'dflt_value':'default_value',"pk": 'primary_key'})
table_info.primary_key = table_info.primary_key.map({1:"Yes" , 0: ""})
table_info

,cid,ColumnName,DataType,notnull,default_value,primary_key
0,0,FID,INTEGER,0,None,Yes
1,1,instructor_fname,TEXT,1,None,
2,2,instructor_lname,TEXT,1,None,


In [33]:
engine = create_engine("sqlite:///CourseData.db")


for i in tables.DatabaseTables.values : 
    
    query = "PRAGMA TABLE_INFO ({}) ".format (i)

    table_info = pd.read_sql_query(query,engine)
    table_info =table_info.rename(columns={"name": "ColumnName" ,"type": "DataType" , 'dflt_value':'default_value',"pk": 'primary_key'})
    table_info.primary_key = table_info.primary_key.map({1:"Yes" , 0: ""})
    print("Table : ", i)
    display(table_info)

Table :  PROGRAMS


,cid,ColumnName,DataType,notnull,default_value,primary_key
0,0,PID,INTEGER,0,None,Yes
1,1,program_code,TEXT,1,None,
2,2,program_name,TEXT,1,None,


Table :  FACULTY


,cid,ColumnName,DataType,notnull,default_value,primary_key
0,0,FID,INTEGER,0,None,Yes
1,1,instructor_fname,TEXT,1,None,
2,2,instructor_lname,TEXT,1,None,


Table :  LOCATION


...

,cid,ColumnName,DataType,notnull,default_value,primary_key
0,0,COID,INTEGER,0,None,Yes
1,1,CID,INTEGER,0,None,
2,2,FID,INTEGER,1,None,
3,3,CRN,TEXT,1,None,
4,4,SEMESTER,TEXT,1,None,
5,5,YEAR,INTEGER,1,None,
6,6,TITLE,TEXT,1,None,
7,7,CATALOG_ID,TEXT,0,None,
8,8,CatalogYear,TEXT,0,None,
9,9,MEETINGS,TEXT,0,None,


Table :  CLASS_MEETINGS


,cid,ColumnName,DataType,notnull,default_value,primary_key
0,0,CMID,INTEGER,0,None,Yes
1,1,COID,INTEGER,0,None,
2,2,LID,INTEGER,0,None,
3,3,DAY,TEXT,0,None,
4,4,START,TEXT,0,None,
5,5,END,TEXT,0,None,
6,6,CRN,TEXT,0,None,
7,7,SEMESTER,TEXT,1,None,
8,8,YEAR,INTEGER,1,None,


Table :  CATALOG_YEAR


,cid,ColumnName,DataType,notnull,default_value,primary_key
0,0,CatalogYear,TEXT,0,None,
1,1,YEAR,TEXT,0,None,
2,2,SEMESTER,TEXT,0,None,
